In [1]:
import pandas as pd
from pymatgen.io.cif import CifParser
from pyxtal import pyxtal
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import os
from pathlib import Path
import zipfile

In [2]:
# file_path = "/home/holywater2/crystal_gen/mattergen/results/sg225_g1e3"
# file_path = "/home/holywater2/crystal_gen/mattergen/results/spg123_base"
file_path = "/home/holywater2/crystal_gen/mattergen/results/"
# file_path += "/spg38_gs0.00001_ge0.001"
file_path += "spg123_gs0.05_ge0.2_v4"

In [3]:
file_path = "/home/holywater2/crystal_gen/mattergen/results/prim/spg6_gs1_ge3_prim_fin/projected"
# file_path = "/home/holywater2/crystal_gen/mattergen/results/prim/spg6_gs5_ge15_prim_fin4/result_0/projected"
# file_path = "/home/holywater2/crystal_gen/mattergen/results/prim/spg38_gs5_ge15_prim_fin2/result_0/projected"

In [4]:
# spg = file_path.split("/")[-1].split("_")[0].replace("spg", "")
# spg = int(spg)
# print(spg)

In [5]:
spg = 38

In [6]:
if not os.path.exists(file_path + "/generated_crystals_cif.zip"):
    print("No zip file found")
elif os.path.exists(file_path + "/generated_crystals_cif"):
    print("Directory already exists")
else:
    zipfile.ZipFile(file_path + "/generated_crystals_cif.zip", 'r').extractall(file_path)
    os.rename(file_path + "/tmp", file_path + "/generated_crystals_cif")
cif_path = file_path + "/generated_crystals_cif"
print(cif_path)

No zip file found
/home/holywater2/crystal_gen/mattergen/results/prim/spg6_gs1_ge3_prim_fin/projected/generated_crystals_cif


In [7]:
# get list of cifs from directory
cif_list = [f for f in os.listdir(cif_path) if f.endswith('.cif')]
cif_list.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))
cif_list = [os.path.join(cif_path, f) for f in cif_list]

FileNotFoundError: [Errno 2] No such file or directory: '/home/holywater2/crystal_gen/mattergen/results/prim/spg6_gs1_ge3_prim_fin/projected/generated_crystals_cif'

In [57]:
structures = []
for cif_file in cif_list:
    parser = CifParser(cif_file)
    print(cif_file)
    structure = parser.get_structures(primitive=False)[0]
    structures.append(structure)

/home/holywater2/crystal_gen/mattergen/results/prim/spg38_gs5_ge15_prim_fin2/result_0/projected/generated_crystals_cif/gen_0.cif


/tmp/ipykernel_2076632/2063553344.py:5: FutureWarning: get_structures is deprecated; use parse_structures in pymatgen.io.cif instead.
The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structure = parser.get_structures(primitive=False)[0]
/home/holywater2/crystal_gen/mattergen/.venv/lib/python3.10/site-packages/pymatgen/io/cif.py:1094: UserWarning: Some occupancies ([2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.0
  warnings.warn(msg)
/home/holywater2/crystal_gen/mattergen/.venv/lib/python3.10/site-packages/pymatgen/io/cif.py:1300: UserWarning: No structure parsed for section 1 in CIF.
Occupancy 2.0 exceeded tolerance.
  warnings.warn(msg)
/home/holywater2/crystal_gen/mattergen/.venv/lib/python3.10/site-packages/pymatgen/io/cif.py:1304: U

ValueError: Invalid CIF file with no structures!

In [14]:
res = {}
for i,structure in enumerate(structures):
    sgn = SpacegroupAnalyzer(structure,symprec=0.1).get_space_group_number()
    # print(i,sgn)
    if sgn not in res:
        res[sgn] = 0
    res[sgn] += 1
for k in sorted(res.keys()):
    print(f"{k:3} {res[k]}  {res[k]/len(structures):.2f}")
print()

if spg in res:
    print(f"{spg} {res[spg]}  {res[spg]/len(structures):.2f}")
else:
    print(f"spg{spg} is not found")
    res[spg] = 0

SymmetryUndeterminedError: 

In [15]:
res = {}
for i,structure in enumerate(structures):
    sgn = SpacegroupAnalyzer(structure,symprec=0.01).get_space_group_number()
    # print(i,sgn)
    if sgn not in res:
        res[sgn] = 0
    res[sgn] += 1
for k in sorted(res.keys()):
    print(f"{k:3} {res[k]}  {res[k]/len(structures):.2f}")
print()

if spg in res:
    print(f"{spg} {res[spg]}  {res[spg]/len(structures):.2f}")
else:
    print(f"spg{spg} is not found")
    res[spg] = 0

  1 27  0.31
  6 61  0.69

spg38 is not found


In [13]:
# write the number of spg
with open(file_path + f"/spg{spg}_{res[spg]}_{len(structures)}.txt", "w") as f:
    for k in sorted(res.keys()):
        f.write(f"{k:3} {res[k]}  {res[k]/len(structures):.2f}\n")
    f.write("\n")
    if spg in res:
        f.write(f"{spg} {res[spg]}  {res[spg]/len(structures):.2f}\n")
    else:
        f.write(f"spg{spg} is not found\n")